In [1]:
import netgen.gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import matplotlib.pyplot as plt

# Example 5.3

We continue with the setting of the previous example. Now let $a(u, v):=\int_{\Omega} 6 \nabla u \nabla v \mathrm{d} x$ and $f(v):=\int_{\Omega} 16 v \mathrm{d} x .$ 

We consider the discrete problem:
Find $u_{h} \in V_{h}$ so that $a\left(u_{h}, v_{h}\right)=f\left(v_{h}\right)$ for all $v_{h} \in V_{h}$

(c) Compute the element matrix $A_{T}$ and the element vector $f_{T}$ for $T_{2} .$ Note, that the basis functions on one particular element are given by the composition of the basis functions on the reference element and the corresponding mapping (Lagrange Finite elements are equivalent!). This means for $\varphi_{m}:=\hat{\varphi}_{m} \circ F_{2}^{-1}$ with $m \in\{0,1,2,3\}$ compute $\left(A_{T}\right)_{m n}:=\int_{T_{2}} 6 \nabla \varphi_{n} \nabla \varphi_{m} \mathrm{d} x$ and $\left(f_{T}\right)_{m}:=\int_{T_{2}} 16 \varphi_{m} \mathrm{d} x, m, n \in\{0,1,2,3,\}$
Document the calculation process.

(d) Give the connectivity matrix $C_{T}$ for element $T_{2}$

(e) In this concrete example it turns out that all element matrices are the same. Set
up the system matrix $A$ and the vector $b$. You can also do this with NGSolve using the prepared file assemble_quads.py where the element matrices and vectors are computed. Note, that you do not have to calculate the connectivity matrices for that.

Finally we want to solve the problem, but want to impose Dirichlet boundary conditions on the whole boundary. Find $u_{h} \in V_{h, D}:=\left\{u_{h} \in V_{h}\left|u_{h}\right|_{\partial \Omega}=1\right\}$ so that $a\left(u_{h}, v_{h}\right)=f\left(v_{h}\right)$
for all $v_{h} \in V_{h, 0}=\left\{u_{h} \in V_{h}\left|u_{h}\right|_{\partial \Omega}=0\right\}$

(f) Solve the (discrete) inhomogeneous Dirichlet problem and make a sketch of the solution $u_{h}(x, y)$

---

Hints:
. Note that in the sketch the local numbering of degrees of freedom (basis function numbering, functional numbering) is indicated inside the element while the global numbering is put on the vertices of the triangulation. Thus for example the global functional $\psi_{4}$ equals the local functional $\psi_{T_{0}}^{1}$ and $\psi_{T_{1}}^{0}$
In assemble_quads.py the element matrices for the same problem are computed and shown. You may want to use this to verify your computations in (c).

In [3]:
# Taken from "assemble_quads.py"
mesh = Mesh("quads.vol.gz")
V = H1(mesh, order=1)
u, v = V.TnT()
bfi =  BilinearForm(V)
bfi += 6*InnerProduct(grad(u), grad(v))
lfi = LinearForm(V)
lfi += 16*u
#bfi = SymbolicBFI(6*grad(V.TrialFunction())*grad(V.TestFunction()))
#lfi = SymbolicLFI(16*V.TrialFunction())

#storage to gather the global matrix:
a = Matrix(V.ndof,V.ndof)
for i in range(V.ndof):
    for j in range(V.ndof):
        a[i,j] = 0.0;

#storage to gather the global vector:
f = Vector(V.ndof)
for i in range(V.ndof):
    f[i] = 0.0

for el in V.Elements():
    print("---------------------------------------- ")
    print("currently on element: ", el.nr)
    print("the vertices of this elements are:\n", el.vertices)
    print("the degrees of freedom of this element are:\n", el.dofs)
    
    dofs = el.dofs
    elmat = bfi.CalcElementMatrix(el.GetFE(),el.GetTrafo())
    print("the element matrix of this element is:\n", elmat)
    elvec = lfi.CalcElementVector(el.GetFE(),el.GetTrafo())
    print("the element vector of this element is:\n", elvec)
    
print("the final assembled matrix is:\n",a)
print("the final assembled vector is:\n",f)

u = GridFunction(V)
u.vec[0:8]=1
u.vec[8]=2.5

print(u.vec)

Draw(u,mesh,"u")

TypeError: __iadd__(): incompatible function arguments. The following argument types are supported:
    1. (self: ngsolve.comp.BilinearForm, other: ngsolve.fem.BFI) -> ngsolve.comp.BilinearForm
    2. (self: ngsolve.comp.BilinearForm, arg0: ngsolve.comp.SumOfIntegrals) -> ngsolve.comp.BilinearForm
    3. (self: ngsolve.comp.BilinearForm, arg0: ngsolve.comp.Variation) -> ngsolve.comp.BilinearForm

Invoked with: <ngsolve.comp.BilinearForm object at 0x00000179754A38B0>, <ngsolve.fem.CoefficientFunction object at 0x0000017975029348>

In [ ]:
### Definition of FES ###
# a)
BND = 'b|r|t|l'
orders = [1,2,3]
ts = [0.1, 0.01, 0.001]
fc = 1
Norms_all = [[],[],[]]

for order, Norms in zip(orders, Norms_all):
    Vw = H1(mesh, order = order, dirichlet=BND)
    Vb = VectorH1(mesh, order = order, dirichlet=BND)

    #This defines a "compound FESpace"
    V = FESpace([Vw, Vb])
    for t in ts:
        t2 = t*t
        # Trial and test functions
        (w, beta) = V.TrialFunction()
        (v, delta) = V.TestFunction()

        #help(beta)

        # BLF
        B = BilinearForm(V, symmetric=True)
        B += (InnerProduct(grad(beta), grad(delta)) + 1/t2 * InnerProduct(grad(w) - beta, grad(v)-delta) )*dx
        # LF
        F = LinearForm(V)
        F += fc*v*dx

        # solution on V
        sol = GridFunction(V, 'sol_mixed')
        w_sol = sol.components[0]
        beta_sol = sol.components[1]

        ### Solving the system ###
        # Assemble the system of equations
        with TaskManager(): # with Multithreading
            F.Assemble()
            B.Assemble()

        # Solve the system
        #sol.vec.data = B.mat.Inverse(V.FreeDofs(), inverse = "umfpack") * F.vec
        sol.vec.data = B.mat.Inverse(V.FreeDofs(), inverse = 'sparsecholesky') * F.vec

        ### Calculating stuff in the system ###

        # L2-norm of grad(w_sol) - beta_sol ... don't know how to do it directly with ngsolve
        norm = Integrate(InnerProduct(grad(w_sol) - beta_sol, grad(w_sol) - beta_sol), mesh)
        Norms.append(norm)
        #test = Integrate(Norm(grad(w_sol) - beta_sol), mesh)
        print(test)
        #print("Norm(grad(w) - beta)=", norm)
        #print('-'*30)
#-------------------------------------------------------------------------------------------#

print("Trying to draw what you cobbled together here...")
Draw(w_sol, mesh, 'w_B1')
Draw(beta_sol, mesh, 'beta_B1')

fig, ax = plt.subplots(1, figsize= [10,6])
ax.loglog(ts, Norms_all[0], 'x-r', label='k=1')
ax.loglog(ts, Norms_all[1], 'x-b', label='k=2')
ax.loglog(ts, Norms_all[2], 'x-g', label='k=3')
ax.set_xlabel('t')
ax.set_ylabel('$||grad(\omega) - \\beta||_{L2}$')
ax.grid(True)
fig.show()

In [ ]:
print("Welcome, Peter!")
### Definition of mesh ###
h = 0.1
geo = SplineGeometry()

geo.AddRectangle((0,0), (1,1), bcs=['b','r','t','l'], leftdomain=1, rightdomain=0)
geo.SetMaterial(1, "D")
mesh = Mesh(geo.GenerateMesh(maxh=h)) # standard mesh available
print('#'*40)
print("Mesh generated:")
print('#'*40)
print("Number of vertices: ", mesh.nv)
print("Number of elements: ", mesh.ne)
print("Dimension of mesh: ", mesh.dim)
Draw(mesh)

In [ ]:
### Definition of FES ###
# b)
BND_b = 't|l'
orders = [1,2,3]
ts = [0.1, 0.01, 0.001]
fc = 1
Norms_all = [[],[],[]]

for order, Norms in zip(orders, Norms_all):
    Vw = H1(mesh, order = order, dirichlet=BND)
    Vb = VectorH1(mesh, order = order, dirichlet=BND)

    #This defines a "compound FESpace"
    V = FESpace([Vw, Vb])
    for t in ts:
        t2 = t*t
        # Trial and test functions
        (w, beta) = V.TrialFunction()
        (v, delta) = V.TestFunction()

        #help(beta)

        # BLF
        B = BilinearForm(V, symmetric=True)
        B += (InnerProduct(grad(beta), grad(delta)) + 1/t2 * InnerProduct(grad(w) - beta, grad(v)-delta) )*dx
        # LF
        F = LinearForm(V)
        F += fc*v*dx

        # solution on V
        sol = GridFunction(V, 'sol_mixed')
        w_sol = sol.components[0]
        beta_sol = sol.components[1]

        ### Solving the system ###
        # Assemble the system of equations
        with TaskManager(): # with Multithreading
            F.Assemble()
            B.Assemble()

        # Solve the system
        #sol.vec.data = B.mat.Inverse(V.FreeDofs(), inverse = "umfpack") * F.vec
        sol.vec.data = B.mat.Inverse(V.FreeDofs(), inverse = 'sparsecholesky') * F.vec

        ### Calculating stuff in the system ###

        # L2-norm of grad(w_sol) - beta_sol ... don't know how to do it directly with ngsolve
        norm = Integrate(InnerProduct(grad(w_sol) - beta_sol, grad(w_sol) - beta_sol), mesh)
        Norms.append(norm)
        #print("Norm(grad(w) - beta)=", norm)
        #print('-'*30)
#-------------------------------------------------------------------------------------------#

print("Trying to draw what you cobbled together here...")
Draw(w_sol, mesh, 'w_B2')
Draw(beta_sol, mesh, 'beta_B2')

fig, ax = plt.subplots(1, figsize= [10,6])
ax.loglog(ts, Norms_all[0], 'x-r', label='k=1')
ax.loglog(ts, Norms_all[1], 'x-b', label='k=2')
ax.loglog(ts, Norms_all[2], 'x-g', label='k=3')
ax.set_xlabel('t')
ax.set_ylabel('$||grad(\omega) - \\beta||_{L2}$')
ax.grid(True)
fig.show()

In [ ]:
### Definition of FES ###
# c)
BND_c = 'b|r|t|l'
orders = [1,2,3]
ts = [0.1, 0.01, 0.001]
fc = 1
Norms_all = [[],[],[]]

for order, Norms in zip(orders, Norms_all):
    Vw = H1(mesh, order = order, dirichlet=BND)
    Vb = VectorH1(mesh, order = order, dirichlet=BND)

    #This defines a "compound FESpace"
    V = FESpace([Vw, Vb])
    for t in ts:
        t2 = t*t
        # Trial and test functions
        (w, beta) = V.TrialFunction()
        (v, delta) = V.TestFunction()

        #help(beta)

        # BLF
        B = BilinearForm(V, symmetrix=True)
        B += (InnerProduct(grad(beta), grad(delta)) + 1/t2 * InnerProduct(grad(w) - beta, grad(v)-delta) )*dx
        # LF
        F = LinearForm(V)
        F += fc*v*dx

        # solution on V
        sol = GridFunction(V, 'sol_mixed')
        w_sol = sol.components[0]
        beta_sol = sol.components[1]

        ### Solving the system ###
        # Assemble the system of equations
        with TaskManager(): # with Multithreading
            F.Assemble()
            B.Assemble()

        # Solve the system
        #sol.vec.data = B.mat.Inverse(V.FreeDofs(), inverse = "umfpack") * F.vec
        sol.vec.data = B.mat.Inverse(V.FreeDofs(), inverse = 'sparsecholesky') * F.vec

        ### Calculating stuff in the system ###

        # L2-norm of grad(w_sol) - beta_sol ... don't know how to do it directly with ngsolve
        norm = Integrate(InnerProduct(grad(w_sol) - beta_sol, grad(w_sol) - beta_sol), mesh)
        Norms.append(norm)
        #print("Norm(grad(w) - beta)=", norm)
        #print('-'*30)
#-------------------------------------------------------------------------------------------#

print("Trying to draw what you cobbled together here...")
Draw(w_sol, mesh, 'w_B3')
Draw(beta_sol, mesh, 'beta_B3')

fig, ax = plt.subplots(1, figsize= [10,6])
ax.loglog(ts, Norms_all[0], 'x-r', label='k=1')
ax.loglog(ts, Norms_all[1], 'x-b', label='k=2')
ax.loglog(ts, Norms_all[2], 'x-g', label='k=3')
ax.set_xlabel('t')
ax.set_ylabel('$||grad(\omega) - \\beta||_{L2}$')
ax.grid(True)
fig.show()